In [1]:
import os
import sys
import tqdm
import pandas as pd

In [2]:
REBADD_LIB_PATH = os.path.abspath(os.pardir)
if REBADD_LIB_PATH not in sys.path:
    sys.path = [REBADD_LIB_PATH] + sys.path

from rebadd.evaluate import evaluate_sr_nov_div

In [3]:
## Test data (navitoclax, ABT-737)
filepath_ref = os.path.join(os.pardir, 'data', 'zinc15', 'zinc15_test.txt')
df_ref = pd.read_csv(filepath_ref, header=None)

referece_smiles_iter = df_ref.iloc[:,0].values.tolist()

print(len(referece_smiles_iter))

2


In [4]:
input_dir = 'baseline'

modelnames = [
    'JTVAE',
    'MARS',
    'MolGPT',
    'RationaleRL',
    'ReLeaSE',
]

filenames = [f'smi_after.csv.{num}' for num in range(10)]

In [5]:
frames = []

for modelname in modelnames:
    
    for filename in filenames:
        
        filepath = os.path.join(input_dir, modelname, filename)
        
        df = pd.read_csv(filepath)
        
        df = df.loc[:,('smiles', 'bcl2', 'bclxl', 'bclw')]
        df.loc[:,'model'] = modelname
        df.loc[:,'checkpoint'] = filename.split('.')[-1]
        
        frames.append(df)

In [6]:
data = []

for df in tqdm.tqdm(frames):

    s_sr, s_nov, s_div = evaluate_sr_nov_div(df, referece_smiles_iter, 'bcl2_bclxl_bclw')
    
    data.append({'SR':s_sr, 'Nov':s_nov, 'Div':s_div, 'Model':df.loc[0,'model'], 'Ckpt':df.loc[0,'checkpoint']})

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:58<00:00,  1.18s/it]


In [7]:
df_records = pd.DataFrame(data)
df_records.loc[:,'HMean'] = (df_records.loc[:,'SR'] * df_records.loc[:,'Nov'] * df_records.loc[:,'Div']) ** 0.333

df_records

,SR,Nov,Div,Model,Ckpt,HMean
0,0.000200,1.0,0.000000,JTVAE,0,0.000000
1,0.000600,1.0,0.000000,JTVAE,1,0.000000
2,0.000200,1.0,0.000000,JTVAE,2,0.000000
3,0.000400,1.0,0.000000,JTVAE,3,0.000000
4,0.000000,0.0,0.000000,JTVAE,4,0.000000
5,0.000600,1.0,0.000000,JTVAE,5,0.000000
6,0.000400,1.0,0.000000,JTVAE,6,0.000000
7,0.000200,1.0,0.000000,JTVAE,7,0.000000
8,0.000400,1.0,0.000000,JTVAE,8,0.000000
9,0.000400,1.0,0.000000,JTVAE,9,0.000000


In [8]:
df_records.groupby('Model').mean(numeric_only=True)

,SR,Nov,Div,HMean
Model,,,,
JTVAE,0.000340,0.9,0.000000,0.000000
MARS,0.121200,1.0,0.907815,0.479499
MolGPT,0.000537,1.0,0.607643,0.058352
RationaleRL,0.342634,1.0,0.830946,0.658121
ReLeaSE,0.000229,0.7,0.144425,0.018291


In [9]:
df_records.groupby('Model').std(numeric_only=True)

,SR,Nov,Div,HMean
Model,,,,
JTVAE,0.000190,0.316228,0.000000,0.000000
MARS,0.003300,0.000000,0.000303,0.004353
MolGPT,0.000279,0.000000,0.422914,0.040840
RationaleRL,0.005235,0.000000,0.000848,0.003352
ReLeaSE,0.000207,0.483046,0.244225,0.029646
